In [ ]:
# example coder 1
class simple_text_coder:
    def write(out_stream, x_list):
        for x in x_list:
            out_stream.write(str(x).encode('utf8'))
            out_stream.write(b'\t')

    def read(inp_stream):
        return list(map(int, inp_stream.read().split()))
    

import io
output = io.BytesIO()
simple_text_coder.write(output, [1, 2, 3, 300])
print(output.getvalue())

In [ ]:
# example coder 2
import struct

class simple_4byte_coder:
    def write(out_stream, x_list):
        for x in x_list:
            out_stream.write(struct.pack('I', x))

    def read(inp_stream):
        res = []
        while True:
            data = inp_stream.read(struct.calcsize('I'))
            if len(data) == 0:
                break
            res.append(struct.unpack('I', data)[0])
        return res

import io
output = io.BytesIO()
simple_4byte_coder.write(output, [1, 2, 3, 300])
print(output.getvalue())

1. Предлагается реализовать varint

In [ ]:
# code here
class varint_coder:
    def write(out_stream, x_list):
        pass

    def read(inp_stream):
        pass

    
import io        
output = io.BytesIO()
varint_coder.write(output, [1, 2, 3, 300])
print(output.getvalue())    

In [ ]:
import io
import random

# test engine
def test_coder_single(coder, x, debug=False):
    output = io.BytesIO()
    coder.write(output, x)
    output_bytes = output.getvalue()
    input = io.BytesIO(output_bytes)
    y = coder.read(input)
    if x != y or debug:
        print('%s -> %s -> %s' % (x, output_bytes, y))    
    return len(output_bytes)


def test_coder(coder, debug):
    total_size = 0.0
    
    test_range = list(range(20000) if not debug else range(0, 300, 29))
    
    # test single ints
    for x in test_range:
        total_size += test_coder_single(coder, [x], debug) / (x + 1)
    print('Done! Size: %f (smaller is better)' % total_size)
    
    # test list of ints
    sample_max_size = 5
    for i in range(len(test_range)):
        sample_size = random.randint(1, sample_max_size)
        x = random.sample(test_range, sample_size)
        total_size += test_coder_single(coder, x, debug)
    print('Done! Size: %f (smaller is better)' % total_size)
    
    
test_coder(simple_text_coder, False)
test_coder(simple_4byte_coder, False)
#test_coder(varint_coder, False)

In [ ]:
import tarfile
from tqdm import tqdm

#with tarfile.open("texts.norm.tar.bz2", "r:bz2") as tar:
with tarfile.open("texts.small.norm.tar.bz2", "r:bz2") as tar:
    for docid, tarinfo in tqdm(enumerate(tar)):
        with tar.extractfile(tarinfo) as inp:
            for term in inp.read().decode('utf8').split():
                # build inverted index in memory
                pass

2. Предлагается сравнить сумарный размер хранения постинглистов разным кодированием:
 * Без сжатия: 4 байта на число
 * varint
 * varint + delta-кодирование
 * bz2
 * bz2 + delta-кодирование

3. Предлагается получить профит за переупорядычивание документов